In [ ]:
from IPython.display import display, Javascript
import json

display(Javascript("""
// Get session ID from URL
const params = new URLSearchParams(window.location.search);
const sessionId = params.get('session');

if (sessionId) {
    // Retrieve code from localStorage
    const code = localStorage.getItem(sessionId);
    if (code) {
        // Inject into first cell and execute
        const cell = Jupyter.notebook.get_cells()[0];
        cell.set_text(code);
        cell.execute();
        localStorage.removeItem(sessionId); // Clean up
    }
}
"""))

# Default imports if no code is found
%matplotlib inline
import control
import matplotlib.pyplot as plt
print("Ready for simulation. Execute cells manually or wait for automatic loading.")